## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.dtypes

City ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Date                    object
dtype: object

In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_max_temps = float(input("Please enter your desired maximum temperature: "))
user_min_temps = float(input("Please enter your desired minimum temperature: "))

In [29]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_pref_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<= user_max_temps) & 
                                        (city_data_df["Max Temp"]>= user_min_temps)]
user_pref_cities_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Tamale,GH,9.4008,-0.8393,82.54,13,1,6.89,clear sky,2022-01-23 20:11:31
1,1,Vaini,TO,-21.2000,-175.2000,81.10,69,9,9.35,clear sky,2022-01-23 20:11:31
4,4,Diamantino,BR,-14.4086,-56.4461,84.18,68,74,3.47,broken clouds,2022-01-23 20:11:32
6,6,Faanui,PF,-16.4833,-151.7500,81.39,72,96,18.03,overcast clouds,2022-01-23 20:11:33
8,8,São Filipe,CV,14.8961,-24.4956,75.18,70,100,11.01,overcast clouds,2022-01-23 20:11:33


In [31]:
# 4a. Determine if there are any empty rows.
print(f'Row Count: {len(user_pref_cities_df)}')
print(f'Null Values: \n{user_pref_cities_df.isnull().sum()}')

Row Count: 268
Null Values: 
City ID                0
City                   0
Country                8
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64


In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
user_pref_dedupe_df = user_pref_cities_df.dropna()
print('\n--- removing Null Values ---\n')
print(f'Recount Null: \n{user_pref_dedupe_df.isnull().sum()}')
user_pref_dedupe_df.head()


--- removing Null Values ---

Recount Null: 
City ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64


,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Tamale,GH,9.4008,-0.8393,82.54,13,1,6.89,clear sky,2022-01-23 20:11:31
1,1,Vaini,TO,-21.2000,-175.2000,81.10,69,9,9.35,clear sky,2022-01-23 20:11:31
4,4,Diamantino,BR,-14.4086,-56.4461,84.18,68,74,3.47,broken clouds,2022-01-23 20:11:32
6,6,Faanui,PF,-16.4833,-151.7500,81.39,72,96,18.03,overcast clouds,2022-01-23 20:11:33
8,8,São Filipe,CV,14.8961,-24.4956,75.18,70,100,11.01,overcast clouds,2022-01-23 20:11:33


In [48]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = user_pref_dedupe_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tamale,GH,82.54,clear sky,9.4008,-0.8393,
1,Vaini,TO,81.10,clear sky,-21.2000,-175.2000,
4,Diamantino,BR,84.18,broken clouds,-14.4086,-56.4461,
6,Faanui,PF,81.39,overcast clouds,-16.4833,-151.7500,
8,São Filipe,CV,75.18,overcast clouds,14.8961,-24.4956,
10,Cape Town,ZA,70.88,clear sky,-33.9258,18.4232,
13,East London,ZA,74.66,clear sky,-33.0153,27.9116,
15,Grand Gaube,MU,79.38,broken clouds,-20.0064,57.6608,
23,Rikitea,PF,78.48,light rain,-23.1203,-134.9692,
24,Comodoro Rivadavia,AR,75.06,scattered clouds,-45.8667,-67.5000,


In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    params['location'] = f'{lat},{lng}'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except:
        print('Hotel not found... Skipping')
        

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [80]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df_clean = hotel_df.copy()
hotel_df_clean = hotel_df_clean.replace(r'^\s*$',float('NaN'),regex=True)
#hotel_df_clean.isnull().sum() #result was 22
hotel_df_clean.dropna(inplace=True)
#hotel_df_clean.isnull().sum() # results was 0
hotel_df_clean.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tamale,GH,82.54,clear sky,9.4008,-0.8393,Tap Hotel
1,Vaini,TO,81.10,clear sky,-21.2000,-175.2000,Keleti Beach Resort
4,Diamantino,BR,84.18,broken clouds,-14.4086,-56.4461,Hotel Delta
6,Faanui,PF,81.39,overcast clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
8,São Filipe,CV,75.18,overcast clouds,14.8961,-24.4956,Hotel Xaguate


In [81]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df_clean.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# Additional cell for when script is closed and reopened, to utilize csv as DF instead of repeating API calls
# hotel_df_clean = pd.read_csv('WeatherPy_vacation.csv')
# hotel_df_clean.head()

In [3]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_clean[["Lat", "Lng"]]

In [7]:

# 11a. Add a marker layer for each city to the map. 
locations = hotel_df_clean[["Lat", "Lng"]]
max_temp = hotel_df_clean["Max Temp"]
fig = gmaps.figure(center=(15.0, 26.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))